In [1]:
%pip install --upgrade --quiet  docx2txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install langchain_community
%pip install -qU langchain-text-splitters

In [22]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
%pip install -qU "langchain-chroma>=0.1.2"

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
%pip install -U langchain langchainhun --quiet

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement langchainhun (from versions: none)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for langchainhun


In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=70,
    chunk_overlap=10
)
loader = Docx2txtLoader('./output.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)
load_dotenv()
embedding = OpenAIEmbeddings(model = 'text-embedding-3-large')


# database = Chroma.from_documents(documents=document_list,collection_name='chroma-food', persist_directory="./chroma", embedding=embedding)
database = Chroma(collection_name='chroma-food', persist_directory="./chroma", embedding_function=embedding)
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model='gpt-4o')

c:\Users\suzuk\AppData\Local\Programs\Python\Python313\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [8]:
asd = database.similarity_search("식당이름 :, 음식 종류 : ", k=100)
asd

[Document(metadata={'source': './output.docx'}, page_content='식당이름:지동관 의정부\n평점:72점\n음식종류:중식,딤섬'),
 Document(metadata={'source': './output.docx'}, page_content='식당이름:용돈 의정부\n평점:73점\n음식종류:돈까스,카츠'),
 Document(metadata={'source': './output.docx'}, page_content='식당이름:일월담 의정부\n평점:74점\n음식종류:간장게장,게장'),
 Document(metadata={'source': './output.docx'}, page_content='식당이름:그때그집 의정부\n평점:77점\n음식종류:삼겹살,순두부'),
 Document(metadata={'source': './output.docx'}, page_content='식당이름:경원식당 의정부\n평점:84점\n음식종류:부대찌개,라면'),
 Document(metadata={'source': './output.docx'}, page_content='식당이름:산오리 의정부\n평점:73점\n음식종류:오리고기,구이'),
 Document(metadata={'source': './output.docx'}, page_content='식당이름:장흥식당 의정부\n평점:73점\n음식종류:부대찌개,미국'),
 Document(metadata={'source': './output.docx'}, page_content='식당이름:오뎅식당 의정부\n평점:81점\n음식종류:부대찌개,부대볶음'),
 Document(metadata={'source': './output.docx'}, page_content='식당이름:다사랑 의정부\n평점:70점\n음식종류:감자탕,볶음밥'),
 Document(metadata={'source': './output.docx'}, page_content='식당이름:우리나라 의정부\n평점:77점\n음식종류:국밥,곰탕'),

In [ ]:


retriever=database.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

dictionary = ["장소를 나타내는 표현 -> 식당 이름"]

prompt = ChatPromptTemplate.from_template(f"""
사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
사전: {dictionary}

질문 : {{question}}
""")
dictionary_chain = prompt | llm | StrOutputParser()

query = "한식이 맛있는 곳을 평점이 높은 순서대로 3개 추천해주세요"

food_chain = {"query" : dictionary_chain} | qa_chain
ai_response = food_chain.invoke({"question" : query})

In [108]:
ai_response

{'query': '한식이 맛있는 식당을 평점이 높은 순서대로 3개 추천해주세요.',
 'result': '1. 그때그집 의정부 (평점: 77점) - 삼겹살, 순두부  \n2. 공릉닭한마리 민락2지구점 (평점: 71점) - 닭한마리, 칼국수  \n3. 닭이봉 의정부점 (평점: 71점) - 쭈꾸미, 쭈삼'}